In [1]:
import numpy as np
from scipy.stats import norm

K = 39
B = 36
T = 0.75
sigma = 0.25
q = 0.01
r = 0.02
S0 = 39

In [2]:
#  
def C(S,K):
    d1 = (np.log(S/K) + (r-q + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S * np.exp(-q*T)* norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def V(S, K):
    a = (r-q)/ sigma ** 2 - 0.5
    return C(S,K) - (B/S) ** (2*a) * C(B*B/S, K)

def V_mc(path, K, B):
    if min(path) > B:
        return max(0, path[-1] - K) * np.exp(-r*T)
    else: 
        return 0

In [5]:
def valuing_down_and_out_calls(n = 1, m = 2 ** 9):
    N = n * m
    dt = T / m
    S = np.zeros((n, m))
    S[:,0] = S0
    for i in range(n):
        for j in range(1, m):
            Z = norm.ppf(np.random.uniform())
            S[i,j] = S[i,j-1] * np.exp(
                (r - q - 0.5 * sigma ** 2 ) * dt + sigma * np.sqrt( dt ) * Z
            )

    V_calc = V(S[0,0],K)
    V_est = np.mean([ V_mc(S[i,:], K , B) for i in range(n)])
    approximation_error = np.abs(V_est - V_calc)
    return [V_calc, V_est, approximation_error]

In [7]:
for k in range(9 + 1):
    n = 50 * 2 ** k
    m = 200
    print(n*m,m,n,valuing_down_and_out_calls(n, m))
    

10000 200 50 [2.321408636702377, 1.605943714908757, 0.71546492179362]
20000 200 100 [2.321408636702377, 2.670485786745106, 0.3490771500427292]
40000 200 200 [2.321408636702377, 2.3532199806612883, 0.03181134395891139]
80000 200 400 [2.321408636702377, 2.4939044023172716, 0.17249576561489466]
160000 200 800 [2.321408636702377, 2.2985051565566366, 0.022903480145740396]
320000 200 1600 [2.321408636702377, 2.292031128955557, 0.02937750774681991]
640000 200 3200 [2.321408636702377, 2.438165144476263, 0.11675650777388613]
1280000 200 6400 [2.321408636702377, 2.4369102347293174, 0.11550159802694049]
2560000 200 12800 [2.321408636702377, 2.446115279013118, 0.12470664231074124]
5120000 200 25600 [2.321408636702377, 2.455422070757107, 0.13401343405472987]


In [23]:
for k in range(9 + 1):
    Nk = 10000 * (2 ** k)
    
    mk = int(np.ceil(Nk ** (1/3) * T ** (2/3)))
    nk = int(np.floor(Nk / mk))

    print(Nk,mk,nk,valuing_down_and_out_calls(nk, mk))

10000 18 555 [2.321408636702377, 2.6739940482273536, 0.3525854115249767]
20000 23 869 [2.321408636702377, 2.5118261664280466, 0.1904175297256696]
40000 29 1379 [2.321408636702377, 2.657520331277599, 0.33611169457522205]
80000 36 2222 [2.321408636702377, 2.446558087530151, 0.12514945082777418]
160000 45 3555 [2.321408636702377, 2.7066813398436538, 0.3852727031412768]
320000 57 5614 [2.321408636702377, 2.5498172595857973, 0.22840862288342034]
640000 72 8888 [2.321408636702377, 2.5493958637043126, 0.22798722700193563]
1280000 90 14222 [2.321408636702377, 2.574262620048839, 0.25285398334646203]
2560000 113 22654 [2.321408636702377, 2.593632857544325, 0.27222422084194786]
5120000 143 35804 [2.321408636702377, 2.434799762383682, 0.11339112568130494]
